In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install validators
!pip install colorama
!pip install transformers

In [ ]:
import os
import sys
#import import_ipynb
sys.path.append('/content/drive/My Drive/HT')
sys.path.append('/content/drive/My Drive/HT/Data')
sys.path.append('/content/drive/My Drive/HT/Dataset/CMU-MultimodalSDK')
sys.path.append('/content/drive/My Drive/HT/Models')
from model_constants import LOADER_PATH

if LOADER_PATH not in sys.path:
    sys.path.append(os.path.abspath(LOADER_PATH))
print(sys.path)
if not os.path.exists(LOADER_PATH):
    print("Check the relative address of data loaders in the constants/paths.py, current address is: ", LOADER_PATH)

from Data.dataloadercreator import get_dataloaders, get_id_word

In [ ]:
train_loader, dev_loader, test_loader = get_dataloaders()
id2word, word2id = get_id_word()

In [ ]:
batch = next(iter(train_loader))

print(batch[0].shape) # word vectors, padded to maxlen
print(batch[1].shape) # visual features
print(batch[2].shape) # acoustic features
#print(batch[3].shape) # labels
#print(batch[4]) # Attention Masks
#print(batch[5]) # lengths

#print(batch[0])
print(' '.join(list(map(lambda x: id2word[x], batch[0].squeeze().detach().numpy()))))

torch.Size([268, 1])
torch.Size([1, 7])
torch.Size([1])
hey guys this is tay here and i'm going to review the very unexciting movie uhh this umm you can call it pretty much an movie you've got uhh patricia arquette i believe yes and she umm plays this woman right here who gets a little possessed in the movie she's a hairdresser then she you know like i said gets possessed and you got all these crazy satanic shenanigans going on umm the plot if you ask me is kind of hollow there's not that much going on you've got like the typical priest guy that's trying to help her out and so forth and he gets caught up in all this stuff maybe even falls in love with her or something like that it's this thing you know i don't really want to go beyond that in uhh describing this plot of the it's just like when you get like living wounds of like the same stuff that christ had so you get like hanging from the arm and the feet and so forth and that she gets to start out with and that's what begins the pos

In [ ]:
import torch
import random
import numpy as np
from tqdm import tqdm_notebook, trange

from transformers import AdamW
from transformers import XLNetTokenizer, XLNetForSequenceClassification


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

#config = XLNetConfig.from_pretrained('xlnet-base-cased')
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=6)
model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
]

# This variable contains all of the hyperparemeter information our training loop needs
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)

cuda


/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSeque

In [ ]:
for step, batch in enumerate(train_loader):
    print("Sentence #", step)
    sentence = ' '.join(list(map(lambda x: id2word[x], batch[0].squeeze().detach().numpy())))
    sentence_tokenized = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt')
    #print(sentence_tokenized)
    print(tokenizer.decode(sentence_tokenized[0]))


In [ ]:
class XLNetEmotionClassification(torch.nn.Module):
  
  def __init__(self, num_labels=6):
    super(XLNetEmotionClassification, self).__init__()
    self.num_labels = num_labels
    self.xlnet = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)
    self.classifier = torch.nn.Linear(6, num_labels)

    torch.nn.init.xavier_normal_(self.classifier.weight)

  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
       
    # last hidden layer
    last_hidden_state = self.xlnet(input_ids, labels=labels)
                                 
    # pool the outputs into a mean vector
    #mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
    logits = self.classifier(last_hidden_state[1])
#     print(logits.view(-1, self.num_labels))
    #logits = logits[:, 1] - logits[:, 0]
    #logits = torch.argmax(logits)
    if labels is not None:
#       loss_fct = BCEWithLogitsLoss()
      print(logits.shape)
      print(labels.shape)
      #input()
      loss = torch.nn.CrossEntropyLoss()(logits, labels)
#       loss = loss_fct(logits.view(-1, self.num_labels),\
#                       labels.view(-1, self.num_labels))
    
      return loss, logits
    else:
      return logits

  def freeze_xlnet_decoder(self):
    """
    Freeze XLNet weight parameters. They will not be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = False
    
  def unfreeze_xlnet_decoder(self):
    """
    Unfreeze XLNet weight parameters. They will be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = True
    
  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    print(preds.shape)
    print(labels.shape)
    print(preds)
    print(labels)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
device='cpu'
model = XLNetEmotionClassification(num_labels=6).to(device)

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  print()
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_loader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    # Unpack the inputs from our dataloader
    #b_input_ids, b_input_mask, b_labels = batch
    ###sentences, _, _, labels, attention_masks, lengths = batch
    sentences, labels, lengths = batch

    print("Sentence #", step)
    sentence = ' '.join(list(map(lambda x: id2word[x], sentences.cpu().squeeze().detach().numpy())))
    sentence_tokenized = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt')
    #print(sentence_tokenized)
    print(tokenizer.decode(sentence_tokenized[0]))

    #print(sentences.T.shape)
    #print(visual.shape)
    #print(acoustic.shape)
    print(labels)
    labels = labels[:, 1:]
    print(labels)
    labels = labels.argmax(dim=1)
    print(labels.shape)
    print(labels)
    
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    
    # Forward pass
    ###outputs = model(sentences.T, attention_mask=attention_masks.T, labels=labels)
    try:
        
        outputs = model(input_ids=sentence_tokenized, labels=labels)
    
        loss = outputs[0]
        print("Loss: ", loss)
        logits = outputs[1]
        print("Logits: ", logits)
        train_loss_set.append(loss.item())    

        # Backward pass
        loss.backward()

        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += sentences.size(0)
        nb_tr_steps += 1
    except RuntimeError as e:
        print("eXPECTION:", sentences.shape, e)
    
    #input()
    

  print("Train loss: {}".format(tr_loss/nb_tr_steps))  
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for step, batch in enumerate(test_loader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    ###sentences, _, _, labels, attention_masks, lengths = batch
    sentences, labels, lengths = batch

    print("Sentence #", step)
    sentence = ' '.join(list(map(lambda x: id2word[x], sentences.cpu().squeeze().detach().numpy())))
    sentence_tokenized = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt')
    #print(sentence_tokenized)
    print(tokenizer.decode(sentence_tokenized[0]))

    #print(sentences.T.shape)
    #print(visual.shape)
    #print(acoustic.shape)
    labels = labels[:, 1:].argmax(dim=1)
    #print(labels.shape)
    #print(attention_masks.T.shape)
    #print(lengths.shape)

    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      ###output = model(sentences.T, attention_mask=attention_masks.T, labels=labels)
      output = XLNetEmotionClassifier(sentence_tokenized, labels=labels)
      logits = output[1]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `men_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForSeque


Sentence # 0
like every young entrepreneur i suppose i was worried about meeting the monthly payroll all of a sudden you understand that you are responsible for the livelihood of many other people not just yourself what was the big turning point when you knew had made it that was probably in the very late when we opened in germany germany is a much bigger market and the concept worked then we thought we could perhaps also go to an even bigger country like japan which had started to use english as their computer language rather than japanese characters that was the first time<sep><cls>
tensor([[1.3333, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[1., 0., 0., 0., 0., 0.]])
torch.Size([1])
tensor([0])
torch.Size([1, 6])
torch.Size([1])
Loss:  tensor(2.8130, grad_fn=<NllLossBackward>)
Logits:  tensor([[-0.9248,  0.6929,  0.2725, -0.3187,  0.1454,  0.0144]],
       grad_fn=<AddmmBackward>)
Sentence # 1
hey guys this is michael got a question for you are you ready to reinvent 

TypeError: ignored

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

for batch in iter(train_loader):
    MAX_LEN = batch[0].shape[0]
    print(MAX_LEN)

    print(batch[0].shape) # word vectors, padded to maxlen
    print(batch[1].shape) # visual features
    print(batch[2].shape) # acoustic features
    print(batch[3].shape)

    index = 0

    for sample in batch[0].T:
        #print(sample[0].shape)
        #print(sample[1].shape)
        #print(sample[2].shape)
        #print(sample[3].shape)
        #print(sample.shape)
        sentence = list(map(lambda x: id2word[x], sample.tolist()))
        sentence = [word for word in sentence if word != '[PAD]']
        sentence = ' '.join(sentence)
        #print(sentence)
        #print(sample[0])
        tokenized_sentence = tokenizer.tokenize(sentence)
        #print(tokenized_sentence)
        inputs = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_sentence]
        #inputs = pad_sequence(inputs, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
        inputs += ['[PAD]'] * (MAX_LEN - len(inputs))
        print(inputs)
        # Create attention masks
        attention_masks = []

        # Create a mask of 1s for each token followed by 0s for padding
        for seq in inputs:
            seq_mask = [float(i != '[PAD]') for i in seq]
            attention_masks.append(seq_mask)

        train_inputs = torch.tensor(inputs)
        train_masks = torch.tensor(attention_masks)
        train_labels = torch.tensor(batch[3][index])
        index += 1

        print(train_inputs.shape)
        print(train_masks.shape)
        print(train_labels.shape)

865
torch.Size([865, 10])
torch.Size([800, 10, 35])
torch.Size([2488, 10, 74])
torch.Size([10, 7])
[17, 2582, 4145, 3848, 3158, 17, 66, 2874, 5292, 40, 331, 24, 1922, 31, 36, 4145, 3848, 3158, 21, 24, 15737, 4145, 3848, 3158, 113, 27, 24, 4145, 3848, 3158, 6719, 4145, 3848, 3158, 331, 24, 27, 886, 61, 1021, 4145, 3848, 3158, 992, 441, 24, 1902, 20, 1439, 4145, 3848, 3158, 3785, 4145, 3848, 3158, 325, 24, 434, 20, 537, 90, 886, 51, 1250, 1439, 359, 4145, 3848, 3158, 248, 4145, 3848, 3158, 53, 44, 5214, 6691, 52, 4145, 3848, 3158, 49, 53, 44, 4145, 3848, 3158, 8619, 52, 4145, 3848, 3158, 9756, 21, 5214, 23021, 41, 995, 332, 40, 231, 86, 21, 40, 4145, 3848, 3158, 176, 464, 80, 71, 1111, 29, 18, 1879, 1578, 44, 22, 216, 135, 62, 154, 21, 2900, 5640, 21, 21, 4145, 3848, 3158, 125, 22, 144, 406, 343, 791, 4145, 3848, 3158, 105, 4145, 3848, 3158, 27, 106, 18468, 4145, 3848, 3158, 161, 5214, 23021, 9756, 4145, 3848, 3158, 21, 48, 4145, 3848, 3158, 2739, 27, 4145, 3848, 3158, 2206, 33, 4145, 38

TypeError: 'int' object is not iterable